# NAVER OPEN API 활용하여 크롤링

## 1. '검색어'에 따른 블로그 제목 & 링크 가져오기

In [3]:
import urllib.request
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# 정보입력
client_id = "*****" # 발급받은 id 입력
client_secret = "**********" # 발급받은 secret 입력
quote = input("검색어를 입력해주세요.: ") #검색어 입력받기
encText = urllib.parse.quote(quote)
display_num = input("검색 출력결과 갯수를 적어주세요.(최대100, 숫자만 입력): ") #출력할 갯수 입력받기
url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&display="+display_num # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
print(url)
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
print(request)
if(rescode==200):
    response_body = response.read()
    #print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

body = response_body.decode('utf-8')

https://openapi.naver.com/v1/search/blog?query=2023%20%EC%8B%A0%EC%A1%B0%EC%96%B4%20%EB%AA%A8%EC%9D%8C&display=10


In [4]:
# 불필요한 ""(큰따옴표)지워주기
body = body.replace('"','')
#body를 나누기
list1 = body.split('\n\t\t{\n\t\t\t')
#naver블로그 글만 가져오기
list1 = [i for i in list1 if 'naver' in i]

In [5]:
list1[0]

"title:<b>2023 신조어 모음<\\/b>,\n\t\t\tlink:https:\\/\\/blog.naver.com\\/azeza\\/223170103447,\n\t\t\tdescription:중요한 건 꺾이지 않는 마음'이라는 제목을 붙여 기사화했는데 이게 커뮤니티에서 화제가 되면서 퍼져나갔다. 여기까지 <b>2023 신조어 모음<\\/b>이었다. #2023신조어 #요즘신조어 #신조어모음 #올해신조어,\n\t\t\tbloggername:유비의 마케팅 인사이트,\n\t\t\tbloggerlink:blog.naver.com\\/azeza,\n\t\t\tpostdate:20230730\n\t\t},"

In [6]:
#블로그 제목, 링크 뽑기
import re
titles = []
links = []
postdates = []
for i in list1:
  title = re.findall('title:(.*?)\n', i)
  link = re.findall('link:(.*?)\n\t\t\tdescription', i)
  postdate = re.findall('postdate:\d{4}\d{2}\d{2}\\n',i)
  titles.append(title)
  links.append(link)
  postdates.append(postdate)
titles = [r[:-1] for i in titles for r in i]
links = [r[:-1] for i in links for r in i]
postdates = [r[:-1] for i in postdates for r in i]

print('<<제목 모음>>')
print(titles)
print('총 제목 수: ',len(titles),'개')#제목갯수확인

print('\n<<링크 모음>>')
print(links)
print('총 링크 수: ',len(links),'개')#링크갯수확인

print('\n<<날짜 모음>>')
print(postdates)
print('총 날짜 수: ',len(postdates),'개')#링크갯수확인

<<제목 모음>>
['<b>2023 신조어 모음<\\/b>', '<b>2023 신조어 모음<\\/b>, 잡채 뜻은?', '<b>2023 신조어 모음<\\/b>! 농협은행 밈 무슨 뜻일까요?', '모르면 손해보는 <b>2023<\\/b> 부동산 <b>신조어 모음<\\/b> 2탄', '<b>2023 신조어 모음<\\/b>, 꾸웨액 뜻? 알아보자!', '요즘 사람 테스트 알 수가 없는 <b>2023 신조어 모음<\\/b> 퀴즈', '<b>2023 신조어<\\/b> 많이 사용하는 단어 <b>모음<\\/b>', '<b>신조어 모음<\\/b>, <b>2023<\\/b>년 MZ세대 따라잡기!', '폼 미쳤다 뜻? <b>2023 신조어 모음<\\/b>!', '<b>2023 신조어 모음<\\/b> 베스트만 정리']
총 제목 수:  10 개

<<링크 모음>>
['https:\\/\\/blog.naver.com\\/azeza\\/223170103447', 'https:\\/\\/blog.naver.com\\/gurwn1725\\/223158156447', 'https:\\/\\/blog.naver.com\\/renedelucy\\/223167804879', 'https:\\/\\/blog.naver.com\\/pobain\\/223132718444', 'https:\\/\\/blog.naver.com\\/19cmall\\/223139936261', 'https:\\/\\/blog.naver.com\\/cosreader\\/223087008223', 'https:\\/\\/blog.naver.com\\/helpbri\\/223151816220', 'https:\\/\\/blog.naver.com\\/luvpp00\\/223119309626', 'https:\\/\\/blog.naver.com\\/hactoefl\\/223048122766', 'https:\\/\\/blog.naver.com\\/ssgh8890\\/223149028756']
총 링크 수:  10 개

<<날짜 모음

In [7]:
# 링크를 다듬기 (필요없는 부분 제거 및 수정)
blog_links = []
for i in links:
    a = i.replace('\\','')
    b = a.replace('?Redirect=Log&logNo=','/')
    blog_links.append(b)

print(blog_links)
print('생성된 링크 갯수:',len(blog_links),'개')

['https://blog.naver.com/azeza/223170103447', 'https://blog.naver.com/gurwn1725/223158156447', 'https://blog.naver.com/renedelucy/223167804879', 'https://blog.naver.com/pobain/223132718444', 'https://blog.naver.com/19cmall/223139936261', 'https://blog.naver.com/cosreader/223087008223', 'https://blog.naver.com/helpbri/223151816220', 'https://blog.naver.com/luvpp00/223119309626', 'https://blog.naver.com/hactoefl/223048122766', 'https://blog.naver.com/ssgh8890/223149028756']
생성된 링크 갯수: 10 개


## 2. 본문 가져오기

In [8]:
#본문 크롤링
import time
import os
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome(os.path.abspath('chromedriver'))
driver.implicitly_wait(3)

#블로그 링크 하나씩 불러서 크롤링
contents = []
for link in blog_links:
  #블로그 링크 하나씩 불러오기
  driver.get(link)
  time.sleep(1)

  #블로그 안 본문이 있는 iframe에 접근하기
  driver.switch_to.frame("mainFrame")

  #본문 내용 크롤링하기
  try:
    a = driver.find_element(By.CLASS_NAME, "se-main-container").text
    contents.append(a)

  except NoSuchElementException:
    a = driver.find_element(By.CSS_SELECTOR, "div#content-area").text
    contents.append(a)
  #print(본문: \n', a)

driver.quit() #창닫기
print("<<본문 크롤링이 완료되었습니다.>>")

C:\Users\daeun\AppData\Local\Temp\ipykernel_28164\846591468.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.abspath('chromedriver'))
Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


<<본문 크롤링이 완료되었습니다.>>


## 3. 크롤링 내용들 DataFrame으로 만들기

In [9]:
#제목, 블로그링크, 본문내용 Dataframe으로 만들기
import pandas as pd
df = pd.DataFrame({'Title':titles, 'Link':blog_links,'Content':contents})
df
#df 저장
# df.to_csv('{}_블로그.csv'.format(quote),encoding='utf-8-sig',index=False)

,Title,Link,Content
0,<b>2023 신조어 모음<\/b>,https://blog.naver.com/azeza/223170103447,2023 신조어 모음\n신조어\n표준어로 등재되지 않은 'New 단어'\n© sin...
1,"<b>2023 신조어 모음<\/b>, 잡채 뜻은?",https://blog.naver.com/gurwn1725/223158156447,"© rossfindon, 출처 Unsplash\n요새는 알쏭달쏭한 신조어들이 꽤 많..."
2,<b>2023 신조어 모음<\/b>! 농협은행 밈 무슨 뜻일까요?,https://blog.naver.com/renedelucy/223167804879,여러분들 안녕하세요!\n늘 유익하고 재미있는 정보 전해드리는\n원더베이프 문정점 인...
3,모르면 손해보는 <b>2023<\/b> 부동산 <b>신조어 모음<\/b> 2탄,https://blog.naver.com/pobain/223132718444,부동산! 모르면 코 베가는 세상?!\n부동산 신조어에는 어떤 말이 있을까? 2탄\n...
4,"<b>2023 신조어 모음<\/b>, 꾸웨액 뜻? 알아보자!",https://blog.naver.com/19cmall/223139936261,요즘 많이 쓰는 신조어 모음 10가지를 소개해드리려고 합니다.\n요즘이라고는 했지만...
5,요즘 사람 테스트 알 수가 없는 <b>2023 신조어 모음<\/b> 퀴즈,https://blog.naver.com/cosreader/223087008223,요즘 사람 테스트는 아무도\n알 수가 없는 2023년 신조어를\n새롭게 경험할 수 ...
6,<b>2023 신조어<\/b> 많이 사용하는 단어 <b>모음<\/b>,https://blog.naver.com/helpbri/223151816220,길 가다보면 젊은 사람이나 초중고등 학생들이 많이 보이는데요.\n길뿐만 아니라 지하...
7,"<b>신조어 모음<\/b>, <b>2023<\/b>년 MZ세대 따라잡기!",https://blog.naver.com/luvpp00/223119309626,"신조어 모음, 2023년 MZ세대 따라잡기!\n안녕하세요! 봄나리입니다. 급변하는 ..."
8,폼 미쳤다 뜻? <b>2023 신조어 모음<\/b>!,https://blog.naver.com/hactoefl/223048122766,"안녕하세요, 해커스유학 블로그지기입니다.\n얼마 전 종영한 '빨간풍선'에서\n나온 ..."
9,<b>2023 신조어 모음<\/b> 베스트만 정리,https://blog.naver.com/ssgh8890/223149028756,생각해 보면 어렸을 때부터 신조어는\n존재해 왔던 것 같습니다.\n하지만 MZ 세대...


In [10]:
df.loc[0].Content

'2023 신조어 모음\n신조어\n표준어로 등재되지 않은 \'New 단어\'\n© sincerelymedia, 출처 Unsplash\n신조어는 새롭게 만들어진 단어 및 용어이나, 표준어로 등재되지 않은 말을 뜻한다. 신조어의 사용자 수가 늘어나고, 자주 언급되면서 다수의 인원이 명확한 의미로 신조어를 사용할 경우 표준어로 인정된다. 하지만 일시적인 유행에 불과하거나 표준어의 조건을 만족시키지 못한다면 그대로 잊힌다.\n2023년,\n새롭게 떠오른 단어는?\n2023 신조어 모음\n그렇다면 2023년에 새롭게 등장한 단어는 어떤 것들이 있을까? 지금부터 알아보자.\n2023 신조어\n1) 점메추\n© hermez777, 출처 Unsplash\n오늘 점메추 해줘\n(자매품: 저메추)\n무슨 뜻일까?\n첫 번째 신조어는 점메추다. 자매품으로 \'저메추\'가 있다. 점메추는 점심메뉴추천의 줄임말로, 점메추 해줘=점심메뉴 추천해달라는 뜻이다. 점심 메뉴 선택은 모든 사람들의 고민인가 보다\n© jaywennington, 출처 Unsplash\n저메추는 저녁메뉴추천의 줄임말이다. 이런 원리면 아침 메뉴 추천은 아메추일 것 같은데, 아메추라는 말은 거의 안 쓰이는 것 같았다.\n2023 신조어\n2) 오뱅알\n두 번째 2023 신조어는 오뱅알이다. 오뱅알은 \'오늘 방송 알찼다\'의 줄임말이다. 한 유명 스트리머의 트위치 방송에서 스트리머가 방송을 \'뱅송\'이라고 찰지게 발음한 데서 유래했다.\n출처: 캐릿\n여담으로 캐릿의 2023 트렌드 능력고사에 이런 문제가 있는데, 처음엔 오픈 뱅킹 알림이나 오! 방금 알았어인 줄 알았다. 저것도 나름 그럴듯한 답변이 아닌가? ㅎㅎ\n2023 신조어\n3) 스불재\n© abhijeet_gourav, 출처 Unsplash\n어쩐지 문화유산의 이름일 것 같은 스불재는 스스로 불러온 재앙의 줄임말로 과한 욕심을 부르다가 쓴맛을 보았을 때 사용하는 용어다. 예시를 들면, 중요한 약속 전 날 새벽에 라면이 땡겨서 먹었는데 다음날 얼굴이 무

# API 사용하지 않고 Selenium으로 크롤링

## 검색어, 옵션 설정 가능

In [11]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller # 크롬 드라이버 자동설치

import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [30]:
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')

In [31]:
# Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome(os.path.abspath('chromedriver'))
# 사이트 주소는 네이버
driver.get('http://www.naver.com')
time.sleep(1)  # 1초간 정지

In [32]:
# Step 2. 네이버 검색창에 "검색어" 검색
from selenium.webdriver.common.by import By
element = driver.find_element(By.ID, "query")
element.send_keys(query_txt)  # query_txt는 위에서 입력한 키워드
element.submit() # 검색어 제출
time.sleep(1)

In [33]:
# 'VIEW' 클릭
driver.find_element(By.LINK_TEXT, '블로그').click()
time.sleep(1)
# '옵션' 클릭
driver.find_element(By.LINK_TEXT, '옵션').click()
time.sleep(1)

In [34]:
item_li = driver.find_elements(By.CSS_SELECTOR,'.option .txt' )
for i in range(0, len(item_li)):
    print(i, item_li[i].text)

0 전체
1 블로그
2 카페
3 관련도순
4 최신순
5 전체
6 1시간
7 1일
8 1주
9 1개월
10 3개월
11 6개월
12 1년
13 직접입력옵션펼치기


In [35]:
# 원하는 옵션 선택해서 클릭!
item_li[9].click()

In [36]:
# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 2  # 스크롤 1번 당 글 30개씩 화면에 보여짐
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1

In [37]:
# URL_raw 크롤링 시작
articles = ".api_txt_lines.total_tit"
article_raw = driver.find_elements(By.CSS_SELECTOR, articles )
#print(article_raw[1].text) # title
print(article_raw[1].get_attribute('href')) # link
driver.quit() # 창닫기


https://blog.naver.com/gurwn1725/223158156447


In [40]:
# 블로그 글 url들 수집
url_list = []
# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')
    if 'blog.naver.com' in url:
        url_list.append(url)
print('url 개수: ', len(url_list))



url 개수:  22


## 크롤링 내용들 DataFrame으로 만들기

In [41]:
df = pd.DataFrame({'url':url_list})

,url
0,https://blog.naver.com/azeza/223170103447
1,https://blog.naver.com/gurwn1725/223158156447
2,https://blog.naver.com/pobain/223153257303
3,https://blog.naver.com/renedelucy/223167804879
4,https://blog.naver.com/notsilly/223153225029
5,https://blog.naver.com/contopia/223172996246
6,https://blog.naver.com/helpbri/223151816220
7,https://blog.naver.com/qkqlqmfkdns32/223169237891
8,https://blog.naver.com/livemoney2007/223163251399
9,https://blog.naver.com/fairy0264/223153346618


- URL에서 제목, 본문 가져오기
    - URL blog id랑 logNo로 넣어야 가져올 수 있네! 
    - 제목을 목록이 아니라 URL로 접속해서 가져오는 이유 : URL 중에서 네이버블로그 아닌 블로그 존재. 네이버 블로그가 아닌 경우 2차링크 규칙이 달라지기 때문

In [48]:
import requests
titles = []
contents = []
for url in df.url:
    #https://blog.naver.com/daddy2015/223169193975	
    blogid= url.split("/")[-2]
    blognum = url.split("/")[-1]
    URL = f'https://blog.naver.com/PostView.naver?blogId={blogid}&logNo={blognum}&redirect=Dlog&widgetTypeCall=true&directAccess=false'
    headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
    response = requests.get(URL, headers = headers)
    print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 제목 가져오기 
    title = soup.select_one("head title ").text
    title = title.replace(' : 네이버 블로그','')
    titles.append(title)

    # 본문 가져오기
    tags = soup.select('.se-main-container .se-component.se-text.se-l-default p span')
    temp = ""
    for tag in tags:
        temp+=tag.text
    contents.append(temp)
    #class="se-component se-text se-l-default"


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [49]:
df['title']= titles
df['content']= contents
df

,url,title,content
0,https://blog.naver.com/azeza/223170103447,2023 신조어 모음,"2023 신조어 모음 신조어는 새롭게 만들어진 단어 및 용어이나, 표준어로 등재되지..."
1,https://blog.naver.com/gurwn1725/223158156447,"2023 신조어 모음, 잡채 뜻은?",요새는 알쏭달쏭한 신조어들이 꽤 많은 것 같아요. 심지어 저는 아직 젊다고(?) 생...
2,https://blog.naver.com/pobain/223153257303,2023년 신조어 모음 2탄,​​알아두면 좋은 신조어 신조어 모음 2탄 야! 너도 신세대? 야! 나도 신세대! ...
3,https://blog.naver.com/renedelucy/223167804879,2023 신조어 모음! 농협은행 밈 무슨 뜻일까요?,​​여러분들 안녕하세요!​늘 유익하고 재미있는 정보 전해드리는원더베이프 문정점 인사...
4,https://blog.naver.com/notsilly/223153225029,"스불재 뜻, 신조어 모음 2023버전 살펴보기(시사상식)","스불재 뜻, 신조어 모음 2023버전 살펴보기이번 포스팅에서는 MZ세대가 많이 사용..."
5,https://blog.naver.com/contopia/223172996246,2023 신조어 테스트로 잘파 세대의 신조어를 R아보자! : 신조어 앤나 밈 테스트 모음,문특 컴눈명 샤이니편 보셨나요?돌아봐 멈춰봐 엠벼 궁합봐야 돼금사빠 레게노 너 T야...
6,https://blog.naver.com/helpbri/223151816220,2023 신조어 많이 사용하는 단어 모음,길 가다보면 젊은 사람이나 초중고등 학생들이 많이 보이는데요.길뿐만 아니라 지하철이...
7,https://blog.naver.com/qkqlqmfkdns32/223169237891,[2023 신조어 테스트와 퀴즈] MZ 용어 줄임말 모음!!,저도 공부 좀 해야겠네요~!!​모두 공부합시다.ㅎㅎ
8,https://blog.naver.com/livemoney2007/223163251399,2023년 요즘 밈 유행어 모음,오늘은 2023년 유행밈과 유행어를알려드리도록 하겠습니다!먼저 밈(MEME)은 민다...
9,https://blog.naver.com/fairy0264/223153346618,"급증하는 신조어, 나도 MZ! 신조어 알아보기",신조어란 새로운 단어라는 뜻입니다. 우리나라에서는 유행어나 줄임말과 같은 의미로 쓰...
